In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import random
import re
from src.functions import *
import random
import datetime

import absl.logging #prevent checkpoint warnings while training
absl.logging.set_verbosity(absl.logging.ERROR)

%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


In [ ]:
obj_library = {}
with open("imagenet1000_clsidx_to_labels.txt") as f:
    obj_library = json.load(f)

In [ ]:
def find_obj_in_text(obj_names,text):

    obj = ""
    for w in re.split("\s|(?<!\d)[,.](?!\d)",text):
        if w in obj_names:
            obj = w
    return obj
def change_obj_names(obj_names,text):
    new_objs_class = random.sample(obj_library.keys(),len(obj_names))
    new_objs = [random.choice(obj_library[o]) for o in new_objs_class]

    obj = find_obj_in_text(obj_names,text)
    if obj != "":
        new_obj = new_objs[obj_names.index(obj)]
        text = text.replace(obj,new_obj)
        obj = new_obj
    return new_objs, obj, text

def load_data(data_dir = "../data/train2", change_names =False):
    
    json_files = [pos_json for pos_json in os.listdir(data_dir) if pos_json.endswith('.json')]
    data = []
    for index, js in enumerate(json_files):
        with open(os.path.join(data_dir, js)) as json_file:
            json_text = json.load(json_file)

            #TODO: change text extraction
            text = js.split("_", 1)[1][:-8] 

            #TODO: change in the normalization
            objs_x = np.array(json_text["meta"]["o_center_x"])/json_text["meta"]["width"]
            objs_y = np.array(json_text["meta"]["o_center_y"])/json_text["meta"]["height"]


            #replace objs for random ones, avoids repetive obj names
            obj_names = json_text["meta"]["obj_names"]
            obj = ""

            if change_names:
                new_objs, obj, text = change_obj_names(obj_names,text)
            else:
                new_objs = obj_names
                obj = find_obj_in_text(obj_names,text)

            
            #TODO: add radius
            data.append({"input_traj":json_text["input_traj"],
                        "output_traj":json_text["output_traj"],
                        "text":text,
                        "obj_names":new_objs,
                        "obj_poses":np.stack([objs_x,objs_y],axis = 0),
                        "obj_in_text":obj
                        })
            # print(data)
            # break

    return data
data = load_data(data_dir = "../data/train", change_names=True) + load_data(data_dir = "../data/train2/train2", change_names=True)

In [ ]:
from src.motion_refiner import Motion_refiner

traj_n = 10
mr = Motion_refiner(traj_n = traj_n)

## ------- processed data -------
X,Y = mr.prepare_data(data,deltas=True)
print("X: ",X.shape)
print("Y: ",Y.shape)

## ------- save pre processed data -------
mr.save_XY(X, Y, x_name="X_delta_new_names",y_name="Y_delta_new_names")
mr.save_data(data,data_name="data_delta_new_names")

# ------- load data --------
# X_, Y_ = mr.load_XY(x_name="X_delta_new_names",y_name="Y_delta_new_names")
# data_ = mr.load_data(data_name="data_delta_new_names")
# feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()